In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error

In [2]:
business_df = pd.read_csv('data/business_data_cleaned.csv')
review_df = pd.read_csv('data/review_data.csv')

In [3]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,business_rating,business_review_count,is_open,Acai Bowls,Accessories,Accountants,Active Life,Adult,Adult Education,Adult Entertainment,Advertising,Afghan,African,Air Duct Cleaning,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Animal Shelters,Antiques,Apartments,Appliances,Appliances & Repair,Aquariums,Arabic,Arcades,Argentine,...,Waxing,Web Design,Wedding Chapels,Wedding Planning,Weight Loss Centers,Wheel & Rim Repair,Whiskey Bars,Wholesale Stores,Wholesalers,Wigs,Wildlife Control,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,good_for_groups,take_out,touristy_ambience,hipster_ambience,romantic_ambience,divey_ambience,intimate_ambience,trendy_ambience,upscale_ambience,classy_ambience,casual_ambience,price_range_1,price_range_2,price_range_3,price_range_4,alcohol_beer_and_wine,alcohol_full_bar,alcohol_no_alcohol
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,North Jefferson Street,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1


In [4]:
review_df.head()

,review_id,user_id,business_id,review_rating
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0


In [5]:
# Scale the `business_review_count` column
scaler = MinMaxScaler()
scaled_features = pd.DataFrame(
    scaler.fit_transform(business_df[['business_review_count']]),
    columns=['business_review_count']
)

# Columns to be dropped from the original dataframe
columns_to_be_dropped = [
    'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'business_review_count'
]

# Drop unnecessary columns including the original `business_review_count`
business_df_cleaned = business_df.drop(columns=columns_to_be_dropped).reset_index(drop=True)

# Combine scaled features with the cleaned dataframe
scaled_business_df = pd.concat([
    business_df_cleaned, 
    scaled_features.reset_index(drop=True)
], axis=1)

# Verify the dataframe
scaled_business_df.head()


,business_id,business_rating,is_open,Acai Bowls,Accessories,Accountants,Active Life,Adult,Adult Education,Adult Entertainment,Advertising,Afghan,African,Air Duct Cleaning,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Animal Shelters,Antiques,Apartments,Appliances,Appliances & Repair,Aquariums,Arabic,Arcades,Argentine,Armenian,Art Classes,Art Galleries,Art Museums,Art Tours,Arts & Crafts,Arts & Entertainment,Asian Fusion,...,Web Design,Wedding Chapels,Wedding Planning,Weight Loss Centers,Wheel & Rim Repair,Whiskey Bars,Wholesale Stores,Wholesalers,Wigs,Wildlife Control,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,good_for_groups,take_out,touristy_ambience,hipster_ambience,romantic_ambience,divey_ambience,intimate_ambience,trendy_ambience,upscale_ambience,classy_ambience,casual_ambience,price_range_1,price_range_2,price_range_3,price_range_4,alcohol_beer_and_wine,alcohol_full_bar,alcohol_no_alcohol,business_review_count
0,MTSW4McQd7CbVtyjqoe9mw,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.009917
1,CF33F8-E6oudUQ46HnavjQ,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.000132
2,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.000661
3,eEOYSgkmpB90uNA7lDOMRA,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.000661
4,il_Ro8jwPlHresjw9EGmBg,2.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0.003041


In [6]:
#Checking if a user has multiple reviews for the same restaurant
print(review_df.shape)
review_df.duplicated(subset=['user_id','business_id']).sum()

(6990280, 4)


244520

We have 244520 duplicate reviews for the same restaurant

In [7]:
aggregated_review_df = review_df.groupby(['user_id', 'business_id'], as_index=False).agg({
    'review_rating': 'mean'
})
aggregated_review_df.shape

(6745760, 3)

In [8]:
#Split train and test data
train_df, test_df = train_test_split(aggregated_review_df,test_size=0.2,random_state=42)
print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")

Train size: 5396608, Test size: 1349152


In [9]:
aggregated_review_df.head()

,user_id,business_id,review_rating
0,---1lKK3aKOuomHnwAkAow,f19eLfhXqR47Ct8Hz2y_pA,5.0
1,---2PmXbF47D870stH1jqA,0ZsqqzHu1HHkDdIKoivi5g,5.0
2,---2PmXbF47D870stH1jqA,1An4DxtMmvvSe0HX4viRCA,5.0
3,---2PmXbF47D870stH1jqA,3YqUe2FTCQr0pPVK8oCv6Q,5.0
4,---2PmXbF47D870stH1jqA,3gXgILE2YWVidJDvVWBT6Q,5.0


In [10]:
scaled_business_df.head()

,business_id,business_rating,is_open,Acai Bowls,Accessories,Accountants,Active Life,Adult,Adult Education,Adult Entertainment,Advertising,Afghan,African,Air Duct Cleaning,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Animal Shelters,Antiques,Apartments,Appliances,Appliances & Repair,Aquariums,Arabic,Arcades,Argentine,Armenian,Art Classes,Art Galleries,Art Museums,Art Tours,Arts & Crafts,Arts & Entertainment,Asian Fusion,...,Web Design,Wedding Chapels,Wedding Planning,Weight Loss Centers,Wheel & Rim Repair,Whiskey Bars,Wholesale Stores,Wholesalers,Wigs,Wildlife Control,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,good_for_groups,take_out,touristy_ambience,hipster_ambience,romantic_ambience,divey_ambience,intimate_ambience,trendy_ambience,upscale_ambience,classy_ambience,casual_ambience,price_range_1,price_range_2,price_range_3,price_range_4,alcohol_beer_and_wine,alcohol_full_bar,alcohol_no_alcohol,business_review_count
0,MTSW4McQd7CbVtyjqoe9mw,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.009917
1,CF33F8-E6oudUQ46HnavjQ,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.000132
2,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.000661
3,eEOYSgkmpB90uNA7lDOMRA,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.000661
4,il_Ro8jwPlHresjw9EGmBg,2.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0.003041


In [11]:
user_business_features = pd.merge(train_df,scaled_business_df,on='business_id')
user_business_features.head()

,user_id,business_id,review_rating,business_rating,is_open,Acai Bowls,Accessories,Accountants,Active Life,Adult,Adult Education,Adult Entertainment,Advertising,Afghan,African,Air Duct Cleaning,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Animal Shelters,Antiques,Apartments,Appliances,Appliances & Repair,Aquariums,Arabic,Arcades,Argentine,Armenian,Art Classes,Art Galleries,Art Museums,Art Tours,Arts & Crafts,...,Web Design,Wedding Chapels,Wedding Planning,Weight Loss Centers,Wheel & Rim Repair,Whiskey Bars,Wholesale Stores,Wholesalers,Wigs,Wildlife Control,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,good_for_groups,take_out,touristy_ambience,hipster_ambience,romantic_ambience,divey_ambience,intimate_ambience,trendy_ambience,upscale_ambience,classy_ambience,casual_ambience,price_range_1,price_range_2,price_range_3,price_range_4,alcohol_beer_and_wine,alcohol_full_bar,alcohol_no_alcohol,business_review_count
0,cJCiN7dtlGLW2-zApamFQA,6N44KZ0qDcY1n1Rz0Nw3bg,3.0,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0.015470
1,viWkK6--5ChiN3hlZ8XqTg,vhDWGF-8BfsxvS7Zo5Wv2w,4.0,4.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0.083961
2,DXY-aDr5b6pdY55AGee2Yg,w1rUV5W_TJ0M0MRwwrseWQ,5.0,4.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0.033849
3,cOeR53mlpxkKQXyD_ucRiQ,gzln_L7ch8z-Ob6autmumw,4.0,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0.028825
4,btzyD8sU1n7dlgHRBECgzw,8fLI8HwccIfQWi0UyH-1Fw,3.5,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0.037683


In [12]:
# Filter for active users and businesses
active_users = user_business_features['user_id'].value_counts()
active_users = active_users[active_users > 10].index  # Users with >10 reviews

active_businesses = user_business_features['business_id'].value_counts()
active_businesses = active_businesses[active_businesses > 10].index  # Businesses with >10 reviews

filtered_user_business_features = user_business_features[
    (user_business_features['user_id'].isin(active_users)) &
    (user_business_features['business_id'].isin(active_businesses))
]

In [13]:
user_profiles_dict = {}
for user_id, group in filtered_user_business_features.groupby('user_id'):
    
    restaurant_features = group.drop(columns=["user_id", "review_rating", "business_id"]).values
    review_ratings = group['review_rating'].values
    user_profile = np.dot(review_ratings,restaurant_features)
    user_profile /= review_ratings.sum()
    user_profiles_dict[user_id] = user_profile
    
user_profiles = pd.DataFrame.from_dict(
    user_profiles_dict, orient='index', columns=scaled_business_df.columns.drop('business_id')
)

user_profiles.head()

,business_rating,is_open,Acai Bowls,Accessories,Accountants,Active Life,Adult,Adult Education,Adult Entertainment,Advertising,Afghan,African,Air Duct Cleaning,Airlines,Airport Lounges,Airport Shuttles,Airport Terminals,Airports,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,Animal Shelters,Antiques,Apartments,Appliances,Appliances & Repair,Aquariums,Arabic,Arcades,Argentine,Armenian,Art Classes,Art Galleries,Art Museums,Art Tours,Arts & Crafts,Arts & Entertainment,Asian Fusion,Auction Houses,...,Web Design,Wedding Chapels,Wedding Planning,Weight Loss Centers,Wheel & Rim Repair,Whiskey Bars,Wholesale Stores,Wholesalers,Wigs,Wildlife Control,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,good_for_groups,take_out,touristy_ambience,hipster_ambience,romantic_ambience,divey_ambience,intimate_ambience,trendy_ambience,upscale_ambience,classy_ambience,casual_ambience,price_range_1,price_range_2,price_range_3,price_range_4,alcohol_beer_and_wine,alcohol_full_bar,alcohol_no_alcohol,business_review_count
---2PmXbF47D870stH1jqA,3.863636,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.363636,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.090909,0.090909,0.0,0.090909,0.0,0.090909,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,0.090909,0.000000,0.000000,0.0,0.272727,0.000000,0.545455,0.636364,0.181818,0.818182,0.000000,0.0,0.272727,0.636364,0.090909,0.124662
--KsuCSkGGvDKTbdK9NvIg,3.877358,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.235849,0.216981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.169811,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.075472,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,0.000000,0.000000,0.056604,0.0,0.075472,0.000000,0.283019,0.830189,0.377358,0.622642,0.000000,0.0,0.396226,0.377358,0.226415,0.065120
--Vu3Gux9nPnLcG9yO_HxA,3.985915,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281690,0.352113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.859155,1.000000,0.0,0.000000,0.070423,0.000000,0.0,0.070423,0.070423,0.492958,0.859155,0.070423,0.929577,0.000000,0.0,0.492958,0.295775,0.211268,0.069814
--_H9j6ggxvqhh9nPofZwg,3.823529,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023529,0.070588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.047059,0.0,...,0.0,0.0,0.0,0.0,0.0,0.035294,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.882353,0.752941,0.0,0.000000,0.000000,0.000000,0.0,0.058824,0.000000,0.270588,0.494118,0.058824,0.764706,0.176471,0.0,0.058824,0.717647,0.223529,0.008044
--_r6E98SNIrGU7weyNxbw,4.015464,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.288660,0.092784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.041237,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.061856,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.845361,1.000000,0.0,0.092784,0.000000,0.000000,0.0,0.103093,0.000000,0.154639,0.443299,0.154639,0.845361,0.000000,0.0,0.051546,0.577320,0.371134,0.049564


In [15]:
# Function to generate recommendations for a user
def recommend_restaurants(user_id, user_profiles, scaled_business_df, top_n=5):
    user_profile = user_profiles.loc[user_id].values.reshape(1, -1)
    business_features = scaled_business_df.drop(columns=["business_id"]).values
    
    # Compute cosine similarity
    similarity_scores = cosine_similarity(user_profile, business_features).flatten()
    
    # Get top N businesses
    top_business_indices = similarity_scores.argsort()[::-1][:top_n]
    recommendations = scaled_business_df.iloc[top_business_indices][["business_id", "business_rating"]]
    return recommendations

# Example: Recommend restaurants for a specific user
user_id = "---2PmXbF47D870stH1jqA"
recommendations = recommend_restaurants(user_id, user_profiles, scaled_business_df)
recommendations

,business_id,business_rating
21049,wbLXXbI-T6Av71i9AoiZRQ,4.5
22095,P6ud6vukerhdIqpoejMFqg,4.5
1836,0qu0fNTOsSmuREYVIMPuIQ,4.5
25147,YbEVMCXzbcibj2a11Yzg1g,4.5
24581,Mbr7CSl0bvDo5kFvZMPv_g,4.0


In [21]:
def precision_at_k(true_ratings, predicted_ratings, k):
    top_k_indices = np.argsort(predicted_ratings)[-k:]  # Get top K indices
    relevant = np.sum([1 for i in top_k_indices if true_ratings[i] >= 4])  # Relevant items have a rating >= 4
    return relevant / k

In [22]:
def mean_average_precision(true_ratings, predicted_ratings):
    sorted_indices = np.argsort(predicted_ratings)[::-1]  # Sort indices by predicted scores in descending order
    relevant_count = 0
    precision_sum = 0.0

    for i, idx in enumerate(sorted_indices):
        if true_ratings[idx] >= 4:  # Define relevance as ratings >= 4
            relevant_count += 1
            precision_sum += relevant_count / (i + 1)

    return precision_sum / relevant_count if relevant_count > 0 else 0.0

In [23]:
def evaluate(df, user_profiles, scaled_business_df, train_or_test, k=5):
    # Convert user profiles and business features to GPU tensors
    user_profiles_tensor = torch.tensor(user_profiles.values, dtype=torch.float32, device='cuda')
    user_profile_index_map = {idx: i for i, idx in enumerate(user_profiles.index)}
    
    business_features_tensor = torch.tensor(scaled_business_df.drop(columns=['business_id']).values, dtype=torch.float32, device='cuda')
    business_id_index_map = {bid: i for i, bid in enumerate(scaled_business_df['business_id'])}

    true_ratings = []
    predicted_ratings = []
    user_predictions = {}
    user_true_ratings = {}

    for _, row in df.iterrows():
        user_id = row['user_id']
        business_id = row['business_id']
        true_rating = row['review_rating']  # Ground truth rating from the train/test set

        # Check if the user and business exist in the respective datasets
        if user_id not in user_profile_index_map or business_id not in business_id_index_map:
            continue  # Skip this entry if either the user or business is missing

        # Retrieve the user's profile tensor
        user_profile_idx = user_profile_index_map[user_id]
        user_profile = user_profiles_tensor[user_profile_idx].unsqueeze(0)

        # Retrieve the business features tensor
        business_features_idx = business_id_index_map[business_id]
        business_features = business_features_tensor[business_features_idx].unsqueeze(0)

        # Compute cosine similarity and predict the rating
        similarity_score = F.cosine_similarity(user_profile, business_features).item()
        predicted_rating = similarity_score

        # Append the results
        true_ratings.append(true_rating)
        predicted_ratings.append(predicted_rating)

        # Store predictions and true ratings per user for MAP and Precision@K
        if user_id not in user_predictions:
            user_predictions[user_id] = []
            user_true_ratings[user_id] = []

        user_predictions[user_id].append(predicted_rating)
        user_true_ratings[user_id].append(true_rating)

    # Calculate RMSE
    if true_ratings and predicted_ratings:
        rmse = torch.sqrt(torch.tensor(mean_squared_error(true_ratings, predicted_ratings), device='cuda')).item()
        if train_or_test == "train":
            print(f"Train RMSE: {rmse}")
        else:
            print(f"Test RMSE: {rmse}")
    else:
        print("No valid predictions were made.")
        return

    # Calculate Precision@K and MAP for all users
    all_precisions = []
    all_maps = []

    for user_id in user_predictions:
        preds = user_predictions[user_id]
        trues = user_true_ratings[user_id]

        # Precision@K
        if len(preds) >= k:
            p_at_k = precision_at_k(trues, preds, k)
            all_precisions.append(p_at_k)

        # MAP
        map_score = mean_average_precision(trues, preds)
        all_maps.append(map_score)

    mean_p_at_k = np.mean(all_precisions) if all_precisions else 0.0
    mean_map = np.mean(all_maps) if all_maps else 0.0

    print(f"Precision@{k}: {mean_p_at_k:.4f}")
    print(f"Mean Average Precision (MAP): {mean_map:.4f}")

In [24]:
evaluate(train_df,user_profiles,scaled_business_df,"train")

Train RMSE: 3.2071684310631343
Precision@5: 0.7846
Mean Average Precision (MAP): 0.8087


In [25]:
evaluate(test_df,user_profiles,scaled_business_df,"test")

Test RMSE: 3.217326563750625
Precision@5: 0.7268
Mean Average Precision (MAP): 0.7970


In [18]:
# Function to generate group recommendations (supports both existing and new users)
def recommend_for_group(group_user_ids, user_profiles, scaled_business_df, business_df, new_user_preferences=None, top_n=5, aggregation_method='average'):
    """
    Recommends restaurants for a group of users, including new users.

    Parameters:
    - group_user_ids: List of user IDs in the group (existing users).
    - user_profiles: DataFrame containing user profiles.
    - scaled_business_df: DataFrame containing scaled business features.
    - business_df: Original DataFrame containing business details.
    - new_user_preferences: Dictionary of preferences for new users.
    - top_n: Number of recommendations to return.
    - aggregation_method: Method to aggregate group preferences ('average', 'min', 'max').

    Returns:
    - DataFrame containing the recommended businesses with their details and similarity scores.
    """
    group_profiles = []

    # Process existing users
    if group_user_ids:
        existing_user_profiles = user_profiles.loc[group_user_ids].values
        group_profiles.extend(existing_user_profiles)

    # Process new user preferences
    if new_user_preferences:
        for user_id, preferences in new_user_preferences.items():
            temp_profile = np.zeros(scaled_business_df.drop(columns=["business_id"]).shape[1])
            for feature, weight in preferences.items():
                if feature in scaled_business_df.columns:
                    temp_profile += weight * scaled_business_df[feature].mean()
            group_profiles.append(temp_profile)

    # Aggregate the profiles to create a group profile
    if len(group_profiles) == 0:
        raise ValueError("No valid user profiles or preferences provided.")
        
    if aggregation_method == 'average':
        group_profile = np.mean(group_profiles, axis=0)
    elif aggregation_method == 'min':
        group_profile = np.min(group_profiles, axis=0)
    elif aggregation_method == 'max':
        group_profile = np.max(group_profiles, axis=0)
    else:
        raise ValueError("Unsupported aggregation method. Use 'average', 'min', or 'max'.")

    # Compute cosine similarity between the group profile and business features
    business_features = scaled_business_df.drop(columns=["business_id"]).values
    similarity_scores = cosine_similarity([group_profile], business_features).flatten()

    # Get the top N business indices
    top_business_indices = similarity_scores.argsort()[::-1][:top_n]

    # Retrieve the business IDs for the top N recommendations
    recommended_business_ids = scaled_business_df.iloc[top_business_indices]["business_id"].values

    # Fetch details from the original business_df
    recommendations = business_df[business_df["business_id"].isin(recommended_business_ids)].copy()
    recommendations['similarity_score'] = similarity_scores[top_business_indices]

    return recommendations[["business_id", "name", "latitude", "longitude", "business_rating", "similarity_score"]]


In [20]:
group_user_ids = ["---2PmXbF47D870stH1jqA", "--KsuCSkGGvDKTbdK9NvIg", "--Vu3Gux9nPnLcG9yO_HxA"]
group_recommendations = recommend_for_group(
    group_user_ids=group_user_ids,
    user_profiles=user_profiles,
    scaled_business_df=scaled_business_df,
    business_df=business_df,
    top_n=5
)
group_recommendations


,business_id,name,latitude,longitude,business_rating,similarity_score
11201,EagkHaaC-kUozD3MPzbRIw,Stanley,29.958250,-90.062923,4.0,0.957344
21049,wbLXXbI-T6Av71i9AoiZRQ,Cleveland-Heath,38.811610,-89.955697,4.5,0.954679
22095,P6ud6vukerhdIqpoejMFqg,Twisted Fork,39.423465,-119.735546,4.5,0.954593
22415,AG8T1jIgBvY0T1G3gkp1Jg,Harvey's 4th Street Grill,27.800496,-82.637700,4.0,0.954590
24581,Mbr7CSl0bvDo5kFvZMPv_g,MeMe's Bar & Grille,29.944299,-89.968667,4.0,0.954060
